In [1]:
#MAC USERS: 
#Before running ipynb - command line:
#ray stop
#ray start --head --port=6379 --redis-password="cbgt2"
#ray start --address='10.162.14.77:6379' --redis-password='cbgt2'
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload 
#if it doesn't work, shut down the ray server and start it again

In [2]:
#!autopep8 --in-place --aggressive --aggressive init_params.py
#!autopep8 --in-place --aggressive --aggressive popconstruct_copy.py

In [3]:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct_copy as popconstruct
import pdb

/home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-07-23 13:29:32,030	INFO worker.py:727 -- Connecting to existing Ray cluster at address: 172.25.0.163:6379


In [4]:
pl = cbgt.Pipeline()

In [5]:
#DEFAULTS - init_params.py
pl.celldefaults = pl[par.helper_cellparams]()
pl.popspecific = pl[par.helper_popspecific]()
pl.receptordefaults = pl[par.helper_receptor]()
pl.basestim = pl[par.helper_basestim]()
pl.dpmndefaults = pl[par.helper_dpmn]()
pl.d1defaults = pl[par.helper_d1]()
pl.d2defaults = pl[par.helper_d2]()
pl.actionchannels = pl[par.helper_actionchannels]()
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

In [6]:
#MODIFIERS - init_params.py 

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)
    
def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)
    
def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)
    
def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)
    
def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)
    
def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self): 
    self.actionchannels = par.helper_actionchannels(self.channels)
    
#POPDATA + PATHWAYS - popconstruct_copy.py

def codeblock_popconstruct_modified(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)
    
def codeblock_poppathways_modified(self): 
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways) 

#CONNECTIVITY GRIDS - popconstruct_copy.py

def codeblock_connAMPA(self):
    self.connectivity_AMPA, self.meaneff_AMPA = popconstruct.helper_connectivityAMPA(self.popdata, self.pathways)

def codeblock_connGABA(self): 
    self.connectivity_GABA, self.meaneff_GABA = popconstruct.helper_connectivityGABA(self.popdata, self.pathways) 

def codeblock_connNMDA(self): 
    self.connectivity_NMDA, self.meaneff_NMDA = popconstruct.helper_connectivityNMDA(self.popdata, self.pathways) 

In [7]:
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct_modified)
pl.add(codeblock_poppathways_modified)
pl.add(codeblock_connAMPA)
pl.add(codeblock_connGABA)
pl.add(codeblock_connNMDA)

In [8]:
pl.modulelist

In [9]:
#DEFAULT VALUES 

environment = {
    'params': None, 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'newpathways' : None
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `p

In [10]:
results.keys()

dict_keys(['params', 'pops', 'receps', 'base', 'dpmns', 'd1', 'd2', 'channels', 'newpathways', 'celldefaults', 'popspecific', 'receptordefaults', 'basestim', 'dpmndefaults', 'd1defaults', 'd2defaults', 'actionchannels', 'popdata', 'pathways', 'connectivity_AMPA', 'meaneff_AMPA', 'connectivity_GABA', 'meaneff_GABA', 'connectivity_NMDA', 'meaneff_NMDA'])

In [11]:
results['pathways']

,src,dest,receptor,type,con,eff,biselector
0,LIP`,D1STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
1,LIP`,D1STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
2,LIP`,D2STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
3,LIP`,D2STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
4,LIP`,FSI`,AMPA`,all`,1.0`,0.198`,<function MatchBiselector.<locals>.Biselector ...
5,LIP`,Th`,AMPA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
6,LIP`,Th`,NMDA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
7,D1STR`,D1STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
8,D1STR`,D2STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
9,D1STR`,GPi`,GABA`,syn`,1.0`,2.09`,<function MatchBiselector.<locals>.Biselector ...


In [12]:
#MODIFIED VALUES - NOT ALL 

environment = {
    'params': pd.DataFrame([[4, 1]], columns=['N', 'C']), 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : pd.DataFrame([[4],[5]], columns=['action']),
    'newpathways' : None
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [13]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [14]:
results['pathways']

,src,dest,receptor,type,con,eff,biselector
0,LIP`,D1STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
1,LIP`,D1STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
2,LIP`,D2STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
3,LIP`,D2STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
4,LIP`,FSI`,AMPA`,all`,1.0`,0.198`,<function MatchBiselector.<locals>.Biselector ...
5,LIP`,Th`,AMPA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
6,LIP`,Th`,NMDA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
7,D1STR`,D1STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
8,D1STR`,D2STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
9,D1STR`,GPi`,GABA`,syn`,1.0`,2.09`,<function MatchBiselector.<locals>.Biselector ...


In [15]:
#MODIFIED VALUES - ALL 

environment = {
    'params': pd.DataFrame([[8, 1]], columns=['N', 'C']),
    'pops' : {'FSI': {'Taum': [60]}}, 
    'receps' : pd.DataFrame([[100, 100]], columns=['Tau_AMPA', 'Tau_GABA']), 
    'base' : {'FSI': {'FreqExt_AMPA': [100]}}, 
    'dpmns' : pd.DataFrame([[400, 100]], columns=['dpmn_wmax', 'dpmn_tauDOP']),
    'd1' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'd2' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'channels' : pd.DataFrame([[8],[9]], columns=['action']),
    'newpathways' : pd.DataFrame([['LIP', 'D1STR', 'AMPA', 'syn', 700, 0.027]], columns=['src', 'dest', 'receptor', 'type', 'con', 'eff'])
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [16]:
results['pathways']

,src,dest,receptor,type,con,eff,biselector
0,LIP`,D1STR`,AMPA`,syn`,700.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
1,LIP`,D1STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
2,LIP`,D2STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
3,LIP`,D2STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
4,LIP`,FSI`,AMPA`,all`,1.0`,0.198`,<function MatchBiselector.<locals>.Biselector ...
5,LIP`,Th`,AMPA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
6,LIP`,Th`,NMDA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
7,D1STR`,D1STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
8,D1STR`,D2STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
9,D1STR`,GPi`,GABA`,syn`,1.0`,2.09`,<function MatchBiselector.<locals>.Biselector ...


In [17]:
#BACK TO DEFAULT VALUES

environment = {
    'params': None, 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'newpathways' : None
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [20]:
results['popdata']

,name,action,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,...,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST,dpmn_type,dpmn_alphaw,dpmn_a,dpmn_b,dpmn_c
0,GPi`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
1,GPi`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
2,STNE`,1`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
3,STNE`,2`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
4,GPeP`,1`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
5,GPeP`,2`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
6,D1STR`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
7,D1STR`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
8,D2STR`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`
9,D2STR`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`


In [21]:
results['pathways']

,src,dest,receptor,type,con,eff,biselector
0,LIP`,D1STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
1,LIP`,D1STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
2,LIP`,D2STR`,AMPA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
3,LIP`,D2STR`,NMDA`,syn`,1.0`,0.027`,<function MatchBiselector.<locals>.Biselector ...
4,LIP`,FSI`,AMPA`,all`,1.0`,0.198`,<function MatchBiselector.<locals>.Biselector ...
5,LIP`,Th`,AMPA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
6,LIP`,Th`,NMDA`,all`,1.0`,0.035`,<function MatchBiselector.<locals>.Biselector ...
7,D1STR`,D1STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
8,D1STR`,D2STR`,GABA`,syn`,0.45`,0.28`,<function MatchBiselector.<locals>.Biselector ...
9,D1STR`,GPi`,GABA`,syn`,1.0`,2.09`,<function MatchBiselector.<locals>.Biselector ...
